<a href="https://www.kaggle.com/code/akashgpt04011995/complete-analysis-ipynb?scriptVersionId=175126218" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-science-challenge-predicting-stock-trends/training_data.csv
/kaggle/input/data-science-challenge-predicting-stock-trends/test_data_no_target.csv
/kaggle/input/data-science-challenge-predicting-stock-trends/group_dictionary.csv
/kaggle/input/data-science-challenge-predicting-stock-trends/column_names_dictionary.csv


In [21]:
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from collections import Counter
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# col_names = pd.read_csv('/kaggle/input/data-science-challenge-predicting-stock-trends/column_names_dictionary.csv', sep='delimiter', header=None)
# print(col_names.shape)
# col_names = col_names[0].str.split(';', n=2, expand=True)
# col_names, col_names.columns = col_names[1:] , col_names.iloc[0]
# col_names.head(2)

### Load Train Data

In [22]:
# Fetching data from csv
train_data = pd.read_csv('/kaggle/input/data-science-challenge-predicting-stock-trends/training_data.csv', sep='delimiter', header=None)
print(train_data.shape)
train_data.head(2)

(8001, 1)


,0
0,Group;I1;I2;I3;I4;I5;I6;I7;I8;I9;I10;I11;I12;I...
1,"G9;0,136495061;-0,02842866;-0,037771783;-0,232..."


### Data Preparation

In [23]:
# Transforming into tabular data
train_data= train_data[0].str.split(';', n=160, expand=True)
train_data, train_data.columns = train_data[1:] , train_data.iloc[0]
train_data.head(2)

,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58,Class,Perform
1,G9,"0,136495061","-0,02842866","-0,037771783","-0,232458668","-0,016221513","-0,187506244","-0,322545081","-0,043742915","0,125388591",...,"0,1025625","0,188480519","-0,016026713","-0,1354512","-0,189667144","0,250966607","0,022171096","-0,004265114",-1,"-0,033763805"
2,G5,"-0,714521622","-0,042137266","-0,052967899","-0,796862406","-0,018393532","0,070102442","-0,076321143","-0,06386426","-1,045520848",...,"-0,016375","0,020727273","-0,006524971","-0,0187904","-0,098542591","0,317744482","-0,180502098","-0,009215046",1,"0,127771413"


In [24]:
# Replacing , with .
train_data = pd.DataFrame({col: train_data[col].str.replace(',', '.') for col in train_data.columns})
train_data.head(2)

,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58,Class,Perform
1,G9,0.136495061,-0.02842866,-0.037771783,-0.232458668,-0.016221513,-0.187506244,-0.322545081,-0.043742915,0.125388591,...,0.1025625,0.188480519,-0.016026713,-0.1354512,-0.189667144,0.250966607,0.022171096,-0.004265114,-1,-0.033763805
2,G5,-0.714521622,-0.042137266,-0.052967899,-0.796862406,-0.018393532,0.070102442,-0.076321143,-0.06386426,-1.045520848,...,-0.016375,0.020727273,-0.006524971,-0.0187904,-0.098542591,0.317744482,-0.180502098,-0.009215046,1,0.127771413


### Convert Categorical Feature to Numerical

In [25]:
# Convert the Group column to a one hot encoded Data Frame
display(train_data['Group'].value_counts())
train_data = pd.get_dummies(train_data, columns=['Group'], drop_first=True, prefix='G')
train_data = train_data.replace({False: 0, True: 1})
# Print the columns names
print(train_data.columns)
train_data.head(2)

Group
G2     1374
G4     1043
G6     1019
G1      880
G5      829
G9      753
G7      677
G8      585
G3      383
G11     297
G10     160
Name: count, dtype: int64

Index(['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10',
       ...
       'G_G10', 'G_G11', 'G_G2', 'G_G3', 'G_G4', 'G_G5', 'G_G6', 'G_G7',
       'G_G8', 'G_G9'],
      dtype='object', length=128)


,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,G_G10,G_G11,G_G2,G_G3,G_G4,G_G5,G_G6,G_G7,G_G8,G_G9
1,0.136495061,-0.02842866,-0.037771783,-0.232458668,-0.016221513,-0.187506244,-0.322545081,-0.043742915,0.125388591,-0.014757394,...,0,0,0,0,0,0,0,0,0,1
2,-0.714521622,-0.042137266,-0.052967899,-0.796862406,-0.018393532,0.070102442,-0.076321143,-0.06386426,-1.045520848,-0.037353088,...,0,0,0,0,0,1,0,0,0,0


In [26]:
train_data = train_data.drop(['Perform'], axis=1)
train_data.shape

(8000, 127)

In [27]:
for col in train_data.columns:
    train_data[col] = pd.to_numeric(train_data[col], errors='coerce')

In [28]:
train_data.dtypes

I1      float64
I2      float64
I3      float64
I4      float64
I5      float64
         ...   
G_G5      int64
G_G6      int64
G_G7      int64
G_G8      int64
G_G9      int64
Length: 127, dtype: object

### Descriptive Analytics

In [ ]:
# sns.violinplot(data=train_data, y='I3')
for col in train_data.columns[:-10]:
    plt.figure(figsize=(7,2))
    q1 = train_data[col].quantile(0.25)
    q3 = train_data[col].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.7*iqr
    df = train_data.loc[(train_data[col] >= fence_low) & (train_data[col] <= fence_high)].copy()
    
    ax = sns.distplot(df[col], bins=100, kde=False)
    ax.axvline(x=fence_low, linewidth=1, color='orange', ls='--')
    ax.axvline(x=fence_high, linewidth=1, color='orange', ls='--')
    print(df.shape)
    plt.show()

### Correlation

In [35]:
correlation_matrix = train_data.corrwith(train_data["Class"]).sort_values(ascending=False).reset_index() # till 55 and -45
correlation_matrix.tail(2)

,index,0
125,I56,-0.047225
126,G_G3,-0.057231


In [36]:
corr_features = correlation_matrix[1:55]['index'].tolist() + correlation_matrix[-45:]['index'].tolist()

### Handle Outliers

### Handle Missing Value

In [37]:
# Check for NaN counts and Blank Values
train_data = train_data[(train_data != 'NA') & (train_data != '')]
# train_data[train_data != ''].isna().sum().values
train_data.isna().sum().values

array([   0,    0,    0,  525,    5,    1,    0,    0,    0,    0,    0,
        526,  145,  675,  523,  262,  190,  150,    0,   32, 1553,  115,
        430,  730,    0,  524,  523,   86,   20,    0,    0,  383,    0,
          0,    0,    0,   32,   32,    0,    0,    0,    0,    7,  150,
        523,  523,    0, 1540,  523, 1540,  523,  523,   11,    0,  133,
         15,  124,  158,    0,    0,    0,  526,   10,    4,    0,    0,
          0,    2,    0,  556,  439,  682,  524,  263,  241,  216,    0,
         33, 1564,  120,  453,  757,    1,  538,  523,  123,   80,    0,
          0,  384,    0,    0,    0,    0,   33,  108,    0,    0,    1,
          0,  214,  473,  523,  523,    0, 1553,  523, 1553,  523,  523,
         15,    0,  170,   20,  135,  249,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [38]:
col_with_missing_vals = []
col_with_minor_missing_vals = []

# summarize the number of rows with missing values for each column
for i in range(train_data.shape[1]):
    # count number of rows with missing values
    n_miss = train_data.iloc[:,i].isna().sum()
    perc = n_miss / train_data.shape[0] * 100
#     print('> %d, %s Missing: %d (%.2f%%)' % (i, train_data.columns[i], n_miss, perc))
    if perc >= 1.00:
        col_with_missing_vals.append(train_data.columns[i])
    elif perc <1.00 and perc >0.00:
        col_with_minor_missing_vals.append(train_data.columns[i])
print(f'{len(col_with_missing_vals)} out of {len(train_data.columns)} have missing values')
print(f'{len(col_with_minor_missing_vals)} out of {len(train_data.columns)} have missing values < 1%')

57 out of 127 have missing values
18 out of 127 have missing values < 1%


In [39]:
# Ist iteration, let's drop all nan rows from columns with < 1% missing values
print(train_data.shape)
train_data = train_data.dropna(subset=col_with_minor_missing_vals)
print(train_data.shape)

(8000, 127)
(7908, 127)


In [40]:
col_with_missing_vals = []
col_with_minor_missing_vals = []

# summarize the number of rows with missing values for each column
for i in range(train_data.shape[1]):
    # count number of rows with missing values
    n_miss = train_data.iloc[:,i].isna().sum()
    perc = n_miss / train_data.shape[0] * 100
    print('> %d, %s Missing: %d (%.2f%%)' % (i, train_data.columns[i], n_miss, perc))
    if perc > 0.00:
        col_with_missing_vals.append(train_data.columns[i])
print(f'{len(col_with_missing_vals)} out of {len(train_data.columns)} have missing values')

> 0, I1 Missing: 0 (0.00%)
> 1, I2 Missing: 0 (0.00%)
> 2, I3 Missing: 0 (0.00%)
> 3, I4 Missing: 488 (6.17%)
> 4, I5 Missing: 0 (0.00%)
> 5, I6 Missing: 0 (0.00%)
> 6, I7 Missing: 0 (0.00%)
> 7, I8 Missing: 0 (0.00%)
> 8, I9 Missing: 0 (0.00%)
> 9, I10 Missing: 0 (0.00%)
> 10, I11 Missing: 0 (0.00%)
> 11, I12 Missing: 489 (6.18%)
> 12, I13 Missing: 144 (1.82%)
> 13, I14 Missing: 636 (8.04%)
> 14, I15 Missing: 486 (6.15%)
> 15, I16 Missing: 260 (3.29%)
> 16, I17 Missing: 190 (2.40%)
> 17, I18 Missing: 149 (1.88%)
> 18, I19 Missing: 0 (0.00%)
> 19, I20 Missing: 0 (0.00%)
> 20, I21 Missing: 1512 (19.12%)
> 21, I22 Missing: 115 (1.45%)
> 22, I23 Missing: 424 (5.36%)
> 23, I24 Missing: 691 (8.74%)
> 24, I25 Missing: 0 (0.00%)
> 25, I26 Missing: 487 (6.16%)
> 26, I27 Missing: 486 (6.15%)
> 27, I28 Missing: 73 (0.92%)
> 28, I29 Missing: 0 (0.00%)
> 29, I30 Missing: 0 (0.00%)
> 30, I31 Missing: 0 (0.00%)
> 31, I32 Missing: 346 (4.38%)
> 32, I33 Missing: 0 (0.00%)
> 33, I34 Missing: 0 (0.00%)


In [ ]:
# # Techinique 1: Fill Missing Vals Using KNN
# # split into input and output elements
# data = train_data.values.copy()
# ix = [i for i in range(data.shape[1]) if i != 116]
# X, y = data[:, ix], data[:,116]
# # print total missing
# print('Missing: %d' % sum(pd.isna(X).flatten()))
# # define imputer
# # imputer = KNNImputer(n_neighbors=3)
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# # fit on the dataset
# imputer.fit(X)
# # transform the dataset
# Xtrans = imputer.transform(X)
# # print total missing
# print('Missing: %d' % sum(pd.isna(Xtrans).flatten()))

### Feature Engineering

In [41]:
def add_calculated_columns(X:pd.DataFrame) -> pd.DataFrame:
    X['profit_ratio'] = np.divide((X['I1'] + X['I2'] + X['I3'] + X['I4']), 4)
    X['liquidity'] = np.divide((X['I50'] + X['I51'] + X['I53']), 3)
    X['leverage'] = np.divide((X['I54'] + X['I55'] + X['I56']), 3)
    X['oper_eff'] = np.divide((X['I22'] + X['I23'] + X['I24'] + X['I25'] + X['I26']), 5)
    X['valuation'] = np.divide((X['I39'] + X['I40'] + X['I41'] + X['I42'] + X['I43']), 5)
    return X
calculated_columns = ['profit_ratio', 'liquidity', 'leverage', 'oper_eff', 'valuation']

In [42]:
print(train_data.shape)
train_data = add_calculated_columns(train_data)
print(train_data.shape)
train_data.head(2)

(7908, 127)
(7908, 132)


,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,G_G5,G_G6,G_G7,G_G8,G_G9,profit_ratio,liquidity,leverage,oper_eff,valuation
1,0.136495,-0.028429,-0.037772,-0.232459,-0.016222,-0.187506,-0.322545,-0.043743,0.125389,-0.014757,...,0,0,0,0,1,-0.040541,0.020126,-0.091364,-0.017685,0.119721
2,-0.714522,-0.042137,-0.052968,-0.796862,-0.018394,0.070102,-0.076321,-0.063864,-1.045521,-0.037353,...,1,0,0,0,0,-0.401622,-0.806741,0.230751,-0.414369,0.003787


In [209]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from xgboost import XGBClassifier

from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE 
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# # define modeling pipeline
# # model = RandomForestClassifier(n_estimators=150, max_depth=26, random_state=1111)
# model = SVC(kernel='rbf')
# # model = GaussianNB()
# # model = AdaBoostClassifier()
# # model = DecisionTreeClassifier()
# # model = LogisticRegression()
# # model = lgb.LGBMClassifier() # .477

# # imputer = KNNImputer()
# pipeline = Pipeline(steps=[('i', imputer), ('m', model)])
# # define model evaluation
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate model
# scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores))) # .474 # RF & ADB .483 # SVC - .487 # NB - .33 # DT - .41

In [ ]:
# # evaluate each strategy on the dataset
# results = list()
# strategies = [str(i) for i in [1,3,7,11,15,19]]
# for s in strategies:
#     # create the modeling pipeline
#     pipeline = Pipeline(steps=[('i', KNNImputer(n_neighbors=int(s))), ('m', SVC(kernel='rbf'))])
#     # evaluate the model
#     cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#     scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
#     # store results
#     results.append(scores)
#     print('>%s %.3f (%.3f)' % (s, np.mean(scores), np.std(scores)))
# # plot model performance for comparison
# plt.boxplot(results, labels=strategies, showmeans=True)
# plt.show()

In [ ]:
# # changing data types to float
# train_data = pd.concat([train_data.iloc[:,0], train_data[train_data.columns[1:]].astype(float)], axis=1)
# train_data.head(2)

### Selecting Corr Features Only

In [44]:
# Spliting input and target features
# train_data['Class'] = train_data['Class'].apply(lambda x:2.0 if x==-1.0 else x)
display(train_data['Class'].value_counts())
X = train_data.drop(['Class'], axis=1)
# X = train_data[corr_features+calculated_columns]
y = train_data['Class'].astype('int')
print(X.shape, y.shape)

Class
 1    3724
-1    3060
 0    1124
Name: count, dtype: int64

(7908, 131) (7908,)


### Train-Test Split

In [227]:
# Splitting train and set data
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.80, random_state=1111, stratify=y)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(6326, 131) (1582, 131) (6326,) (1582,)


### Handle Null Values

In [228]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# fit on the dataset
X_train = imputer.fit_transform(X_train)
# transform the dataset
X_val = imputer.transform(X_val)

### Handle Scaling with Min-Max

In [229]:
scaler = MinMaxScaler()
# fit on the dataset
X_train = scaler.fit_transform(X_train)
# transform the dataset
X_val = scaler.transform(X_val)

### Handle Imbalance Data

In [230]:
sm = SMOTE(random_state = 21) 
print(Counter(y_train))
X_train, y_train = sm.fit_resample(X_train, y_train)
Counter(y_train)

Counter({1: 2979, -1: 2448, 0: 899})


Counter({-1: 2979, 1: 2979, 0: 2979})

## Handle Outliers

In [231]:
clf = IsolationForest(max_samples=500) #, contamination=.01) # , random_state=4
#identify outliers:
y_pred_train = clf.fit_predict(X_train)
#Remove outliers where 1 represent inliers and -1 represent outliers:
X_train_cleaned = X_train[np.where(y_pred_train == 1, True, False)]
y_train_cleaned = y_train[np.where(y_pred_train == 1, True, False)]

### Modelling

In [248]:
# Model Fitting
# Create a random forest classifier
# model = LogisticRegression()
# model = RandomForestClassifier(n_estimators=150, max_depth=26)
# model = SVC(kernel='rbf')
# model = lgb.LGBMClassifier()
# model = KNeighborsClassifier(n_neighbors=180)
model = GaussianNB(var_smoothing=0.071)
scores = cross_val_score(model, X_train_cleaned, y_train_cleaned, cv=3, scoring='f1_macro')
print(np.mean(scores))
# Fit rfc using X_train and y_train
# model.fit(X_train, y_train)
model.fit(X_train_cleaned, y_train_cleaned)
# Create predictions on X_test
preds = model.predict(X_val)
print(preds[0:5])
# Print model accuracy using score() and the testing data
print(model.score(X_val, y_val))

# from sklearn.metrics import f1_score
print(f1_score(y_val, preds, average='macro'))

0.27857454389467656
[0 0 0 0 0]
0.2193426042983565
0.21536138470868774


### Prediction

In [249]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
preds = model.predict(X_val)
cm = confusion_matrix(y_val, preds)
# ConfusionMatrixDisplay(cm, model.classes_).plot()
print("confusion_matrix: ", cm)
cost_matrix = [[0,1,2], [1,0,1], [2,1,0]]
print("cost_matrix: ", cost_matrix)
err = np.sum((cm*cost_matrix)/len(y_val))
print("error: ", err)

confusion_matrix:  [[ 82 489  41]
 [ 34 175  16]
 [ 57 598  90]]
cost_matrix:  [[0, 1, 2], [1, 0, 1], [2, 1, 0]]
error:  0.8426042983565107


### Prediction with XGBoost

In [250]:
from sklearn.preprocessing import LabelEncoder
model = XGBClassifier()
le = LabelEncoder()
y_train_m = le.fit_transform(y_train)
model.fit(X_train, y_train_m)
preds = model.predict(X_val)
preds = le.inverse_transform(preds)

print(f1_score(y_val, preds, average='macro'))
cm = confusion_matrix(y_val, preds)
# ConfusionMatrixDisplay(cm, model.classes_).plot()
print("confusion_matrix: ", cm)
cost_matrix = [[0,1,2], [1,0,1], [2,1,0]]
print("cost_matrix: ", cost_matrix)
err = np.sum((cm*cost_matrix)/len(y_val))
print("error: ", err)

0.36233840336856576
confusion_matrix:  [[274  77 261]
 [ 83  29 113]
 [264 100 381]]
cost_matrix:  [[0, 1, 2], [1, 0, 1], [2, 1, 0]]
error:  0.8994943109987358


In [ ]:
# ### defining the model 
# import xgboost as xgb
# from xgboost import XGBClassifier
# model=xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, tree_method = 'gpu_hist',min_child_weight=5, reg_lambda=20, gamma=2 ,random_state=69,
#                        reg_alpha=26,subsample=0.9,colsample_bytree=0.12,max_depth=30)
# # Fit rfc using X_train and y_train
# model.fit(X_train, y_train)
# # Create predictions on X_test
# preds = model.predict(X_val)
# print(preds[0:5])
# # Print model accuracy using score() and the testing data
# print(model.score(X_val, y_val))

### Prediction on Test Data

In [251]:
# Fetching data from csv
test_data = pd.read_csv('/kaggle/input/data-science-challenge-predicting-stock-trends/test_data_no_target.csv', sep='delimiter', header=None)
print(test_data.shape)
test_data.head(2)

(2001, 1)


,0
0,Group;I1;I2;I3;I4;I5;I6;I7;I8;I9;I10;I11;I12;I...
1,"G3;0,039283602;-0,034382297;-0,040372758;0,090..."


In [252]:
# Transforming into tabular data
test_data= test_data[0].str.split(';', n=160, expand=True)
test_data, test_data.columns = test_data[1:] , test_data.iloc[0]
test_data.head(2)

,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58
1,G3,"0,039283602","-0,034382297","-0,040372758","0,090022863","-0,055952942","0,049228792","0,016236327","-0,045864875","0,692038889",...,"-0,377892086","-0,410908257","-0,4174375","-0,220480519","0,002526132","0,124542933","-0,219341446","0,148324265","0,020379011","-0,004968758"
2,G9,"-0,49171284","-0,039484003","-0,043611749","-0,320311784","0,065520424","-0,216313827","-0,366573195","-0,052790325","-0,370449142",...,"0,553798561","0,660504587","0,806","0,781506494","-0,012519164","-0,014168533","0,669341446","0,015032946","0,023410578","0,00703696"


In [253]:
# Replacing , with .
test_data = pd.DataFrame({col: test_data[col].str.replace(',', '.') for col in test_data.columns})
test_data.head(2)

,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58
1,G3,0.039283602,-0.034382297,-0.040372758,0.090022863,-0.055952942,0.049228792,0.016236327,-0.045864875,0.692038889,...,-0.377892086,-0.410908257,-0.4174375,-0.220480519,0.002526132,0.124542933,-0.219341446,0.148324265,0.020379011,-0.004968758
2,G9,-0.49171284,-0.039484003,-0.043611749,-0.320311784,0.065520424,-0.216313827,-0.366573195,-0.052790325,-0.370449142,...,0.553798561,0.660504587,0.806,0.781506494,-0.012519164,-0.014168533,0.669341446,0.015032946,0.023410578,0.00703696


In [254]:
test_data = pd.get_dummies(test_data, columns=['Group'], drop_first=True, prefix='G')
test_data = test_data.replace({False: 0, True: 1})

In [255]:
for col in test_data.columns:
    test_data[col] = pd.to_numeric(test_data[col], errors='coerce')
print(test_data.dtypes)

I1      float64
I2      float64
I3      float64
I4      float64
I5      float64
         ...   
G_G5      int64
G_G6      int64
G_G7      int64
G_G8      int64
G_G9      int64
Length: 126, dtype: object


In [256]:
# Check for NaN counts and Blank Values
test_data = test_data[(test_data != 'NA') & (test_data != '')]
# train_data[train_data != ''].isna().sum().values
test_data.isna().sum().values

array([  0,   0,   0, 119,   0,   0,   0,   0,   0,   0,   0, 119,  44,
       157, 119,  57,  54,  40,   0,   4, 367,  29, 100, 178,   0, 120,
       119,  17,   2,   0,   0,  82,   0,   0,   0,   0,   4,   4,   0,
         0,   0,   0,   1,  45, 119, 119,   0, 367, 119, 367, 119, 119,
         2,   0,  36,   8,  32,  42,   0,   2,   2, 119,   0,   1,   0,
         2,   0,   2,   0, 121, 105, 158, 119,  57,  66,  57,   0,   6,
       368,  30, 104, 186,   0, 123, 119,  18,  13,   0,   0,  82,   0,
         0,   0,   0,   6,  20,   0,   0,   0,   0,  67, 111, 119, 119,
         0, 368, 119, 368, 119, 119,   3,   0,  46,   9,  34,  67,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])

In [257]:
test_data = add_calculated_columns(test_data)
print(test_data.shape)
test_data.head(2)

(2000, 131)


,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,G_G5,G_G6,G_G7,G_G8,G_G9,profit_ratio,liquidity,leverage,oper_eff,valuation
1,0.039284,-0.034382,-0.040373,0.090023,-0.055953,0.049229,0.016236,-0.045865,0.692039,-0.027972,...,0,0,0,0,0,0.013638,-0.025407,-0.646818,-0.090213,-0.042646
2,-0.491713,-0.039484,-0.043612,-0.320312,0.065520,-0.216314,-0.366573,-0.052790,-0.370449,-0.031982,...,0,0,0,0,1,-0.223780,0.472417,0.156072,-0.075982,-0.115615


In [258]:
X_test = test_data
# X_test = test_data[corr_features+calculated_columns]

In [259]:
# transform the dataset
X_test = imputer.transform(X_test)
X_test = scaler.transform(X_test)

In [260]:
test_preds = model.predict(X_test)
print(test_preds[:5])
print(test_preds.shape)

[0 1 2 2 2]
(2000,)


In [261]:
pd.DataFrame(test_preds).to_csv('test_preds1.csv', index=False)